# Replication of De Bondt and Thaler (1985)

This notebook is adapted from code written by [Charles Martineau](http://www.charlesmartineau.com/) at the University of Toronto.
Prepared by [Vincent Grégoire](http://www.vincentgregoire.com), Department of Finance, The University of Melbourne. 

Objectives:

- Test for the overreaction hypothesis in financial markets based on 
  [De Bondt and Thaler (1985), Does the Stock Market Overreact?,
  Journal of Finance](http://onlinelibrary.wiley.com/doi/10.1111/j.1540-6261.1985.tb05004.x/full).
- Get similar results to Figure 3 in the paper.

Why:

- Research in experimental psychology has suggested that, in violation of
  Baye's rule, most people "overreact" to unexpected and dramatic news
  events. The question then arises whether such behavior matters at the
  market level.
  
This is not meant to be a perfect replication. In fact, I deliberately made different choices in the filtering (number of observations required, listing exchanges, etc.) and in the way to assess performance (computing abnormal returns, forming portfolios, etc.). The goal here is to replicate the spirit, and educate on how one can compute portfolio returns in Python.
 
This notebook was created as supplemental material to a Python for financial research workshop for finance honours and PhD students at the University of Melbourne in March of 2018.

Latest version: <https://github.com/vgreg/python-finance-unimelb2018>

In [2]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime, timedelta

## Step 1: load the CRSP monthly file

The WRDS files uses characters (letters such as `C`) to identify special cases. In our case we'll ignore these, but it's good practice to read the dataset documentation carefully to make sure that what you are doing is reasonable.

In this case, we reload the file, telling pandas to treat `C` as a null value. We also parse the dates.

In [3]:
df_crsp = pd.read_csv(r'C:\Users\10317\Desktop\kw4smgwgvkezomgo.csv.gz', na_values=['C'], parse_dates=['date'])

# The output from WRDS returns a mixed of small and large cap column names.
# We can easily convert everything to small caps.
cols = df_crsp.columns
df_crsp.columns = [c.lower() for c in cols]
df_crsp.drop_duplicates(keep='first',inplace=True)
df_crsp.dropna(inplace=True)
df_crsp.reset_index(inplace=True,drop=True)
df_crsp = df_crsp[df_crsp['exchcd'] == 1.0]
df_crsp

,permno,date,exchcd,permco,prc,vol,ret,shrout,vwretd,ewretd
0,10006,1926-01-30,1.0,22156,110.250,753.0,0.032732,600.0,0.000561,0.023174
1,10006,1926-02-27,1.0,22156,102.375,342.0,-0.071429,600.0,-0.033046,-0.053510
2,10006,1926-03-31,1.0,22156,96.500,489.0,-0.042735,600.0,-0.064002,-0.096824
3,10006,1926-04-30,1.0,22156,94.000,249.0,-0.025907,600.0,0.037029,0.032975
4,10006,1926-05-28,1.0,22156,96.250,81.0,0.023936,600.0,0.012095,0.001035
...,...,...,...,...,...,...,...,...,...,...
966405,93172,1974-08-30,1.0,699,5.500,1711.0,-0.371429,2495.0,-0.087757,-0.085788
966406,93172,1974-09-30,1.0,699,5.000,582.0,-0.090909,2495.0,-0.109718,-0.079025
966407,93172,1974-10-31,1.0,699,3.125,1137.0,-0.375000,2495.0,0.165584,0.088877
966408,93172,1974-11-29,1.0,699,2.625,1031.0,-0.160000,2929.0,-0.041046,-0.051395


In [ ]:
data_1926 = df_crsp[df_crsp['date'].dt.year == 1926]  # 筛选出1926年的数据,这一步是在check有多少上市公司
# 根据 permno 去重，统计公司数量
unique_companies = data_1926['permno'].nunique()
unique_companies

553

In [ ]:
df_crsp['prc'] = abs(df_crsp['prc'])#取绝对值
df_crsp

,permno,date,exchcd,permco,prc,vol,ret,shrout,vwretd,ewretd
0,10006,1926-01-30,1.0,22156,110.250,753.0,0.032732,600.0,0.000561,0.023174
1,10006,1926-02-27,1.0,22156,102.375,342.0,-0.071429,600.0,-0.033046,-0.053510
2,10006,1926-03-31,1.0,22156,96.500,489.0,-0.042735,600.0,-0.064002,-0.096824
3,10006,1926-04-30,1.0,22156,94.000,249.0,-0.025907,600.0,0.037029,0.032975
4,10006,1926-05-28,1.0,22156,96.250,81.0,0.023936,600.0,0.012095,0.001035
...,...,...,...,...,...,...,...,...,...,...
966405,93172,1974-08-30,1.0,699,5.500,1711.0,-0.371429,2495.0,-0.087757,-0.085788
966406,93172,1974-09-30,1.0,699,5.000,582.0,-0.090909,2495.0,-0.109718,-0.079025
966407,93172,1974-10-31,1.0,699,3.125,1137.0,-0.375000,2495.0,0.165584,0.088877
966408,93172,1974-11-29,1.0,699,2.625,1031.0,-0.160000,2929.0,-0.041046,-0.051395


## Step2： 筛选出不同时间点所对应的符合条件的流动性ok的股票

In [21]:
from collections import defaultdict

# 假设 new_data 已经加载，并确保 'date' 列是 datetime 类型
df_crsp['date'] = pd.to_datetime(df_crsp['date'])  # 确保日期是datetime格式
#new_data['date'] = pd.to_datetime(new_data['date'])  # 确保日期是datetime格式
# 初始起始年份和月份
start_year = 1926
start_month = 1

# 初始化变量
date = []
valid_firm_number = []
total_number = []
filter_all_data = defaultdict(list)
# 进行16次循环，每次起始时间向后增加3年
for i in range(16):
    sum_valid = 0  # 累计连续85个月的次数
    total_tests = 0  # 总的测试次数
    # 计算当前循环的起始时间
    start_date = pd.Timestamp(year=start_year + i * 3, month=start_month, day=1)
    end_date = start_date + pd.DateOffset(months=85)  # 85个月的截止时间

    #print(f"\n第 {i+1} 次测试: 起始时间 {start_date.date()}")
    date.append(start_date.date())

    # 筛选在当前时间范围内的数据
    filtered_data = df_crsp[(df_crsp['date']>= start_date)]
    #filtered_data = new_data[(new_data['date']>= start_date)]
    grouped_data = filtered_data.groupby('permno').filter(lambda x: (x['date'].iloc[0].year == start_date.year) and (x['date'].iloc[0].month == start_date.month))
    grouped_data = grouped_data.groupby('permno')  # 再次按 permno 分组

    # 遍历每个分组，判断前85个月的连续性
    for idx, group in grouped_data:
        total_tests += 1  # 统计总测试次数  

        # 提取当前分组前85个月的数据
        first_85 = group.head(85).copy()
        #print(first_85.head())        
        first_85['year'] = first_85['date'].dt.year
        first_85['month'] = first_85['date'].dt.month
        first_85['month_number'] = (first_85['year'] - first_85['year'].iloc[0]) * 12 + first_85['month']

        # 判断连续性
        first_85['month_diff'] = first_85['month_number'].diff().fillna(1)
        first_85['is_consecutive'] = first_85['month_diff'] == 1

        if first_85['is_consecutive'].all():
            #print(f"permno {idx}: 前85个月的数据是连续的。")
            sum_valid += 1
            filter_all_data[start_date.date()].append(idx)
        #else:
            #print(f"permno {idx}: 前85个月的数据存在中断。")
    valid_firm_number.append(sum_valid)
    total_number.append(total_tests)

# 计算并输出结果
for i in range(16):
    print(f"起始日期:{date[i]},符合要求公司比例: {valid_firm_number[i] / total_number[i]:.2%},符合要求公司总数: {valid_firm_number[i]}, 有效公司总数: {total_number[i]}")

起始日期:1926-01-01,符合要求公司比例: 70.91%,符合要求公司总数: 351, 有效公司总数: 495
起始日期:1929-01-01,符合要求公司比例: 72.57%,符合要求公司总数: 455, 有效公司总数: 627
起始日期:1932-01-01,符合要求公司比例: 83.79%,符合要求公司总数: 610, 有效公司总数: 728
起始日期:1935-01-01,符合要求公司比例: 91.51%,符合要求公司总数: 647, 有效公司总数: 707
起始日期:1938-01-01,符合要求公司比例: 96.04%,符合要求公司总数: 751, 有效公司总数: 782
起始日期:1941-01-01,符合要求公司比例: 97.22%,符合要求公司总数: 769, 有效公司总数: 791
起始日期:1944-01-01,符合要求公司比例: 97.91%,符合要求公司总数: 795, 有效公司总数: 812
起始日期:1947-01-01,符合要求公司比例: 99.34%,符合要求公司总数: 902, 有效公司总数: 908
起始日期:1950-01-01,符合要求公司比例: 98.48%,符合要求公司总数: 973, 有效公司总数: 988
起始日期:1953-01-01,符合要求公司比例: 97.80%,符合要求公司总数: 1022, 有效公司总数: 1045
起始日期:1956-01-01,符合要求公司比例: 96.87%,符合要求公司总数: 1020, 有效公司总数: 1053
起始日期:1959-01-01,符合要求公司比例: 95.31%,符合要求公司总数: 1016, 有效公司总数: 1066
起始日期:1962-01-01,符合要求公司比例: 96.23%,符合要求公司总数: 1098, 有效公司总数: 1141
起始日期:1965-01-01,符合要求公司比例: 97.79%,符合要求公司总数: 1196, 有效公司总数: 1223
起始日期:1968-01-01,符合要求公司比例: 98.22%,符合要求公司总数: 1216, 有效公司总数: 1238
起始日期:1971-01-01,符合要求公司比例: 94.21%,符合要求公司总数: 1252, 有效公司总数: 1329


In [33]:
#可以通过如下方式筛选出不同起始点所对应的满足连续85个月都有数值的要求的股票，数字为代码 permno
filter_all_data[pd.to_datetime('1926-01-01').date()]

[10030,
 10049,
 10057,
 10065,
 10081,
 10102,
 10110,
 10129,
 10137,
 10145,
 10188,
 10196,
 10209,
 10217,
 10225,
 10233,
 10241,
 10268,
 10305,
 10313,
 10321,
 10356,
 10364,
 10372,
 10399,
 10401,
 10428,
 10436,
 10444,
 10460,
 10487,
 10495,
 10516,
 10524,
 10559,
 10575,
 10583,
 10591,
 10604,
 10612,
 10647,
 10655,
 10671,
 10698,
 10719,
 10743,
 10751,
 10778,
 10786,
 10794,
 10858,
 10874,
 10890,
 10911,
 10938,
 10946,
 10954,
 10970,
 10989,
 10997,
 11009,
 11025,
 11033,
 11041,
 11068,
 11092,
 11105,
 11113,
 11121,
 11148,
 11156,
 11164,
 11172,
 11199,
 11201,
 11252,
 11260,
 11287,
 11295,
 11308,
 11324,
 11340,
 11359,
 11367,
 11391,
 11404,
 11412,
 11447,
 11463,
 11471,
 11498,
 11500,
 11535,
 11543,
 11551,
 11594,
 11607,
 11615,
 11623,
 11658,
 11666,
 11674,
 11682,
 11690,
 11703,
 11711,
 11746,
 11754,
 11762,
 11789,
 11818,
 11834,
 11850,
 11877,
 11885,
 11914,
 11922,
 11930,
 11949,
 11957,
 11965,
 11973,
 11981,
 12001,
 12028,


In [ ]:
df_crsp.info()

In [ ]:
# Set the index (to select easily on date)
df_crsp = df_crsp.set_index('date')
df_crsp = df_crsp.sort_index()

In [ ]:
# Some data cleaning

# We can drop SHRCD and EXCHCD, we already the filtering.
del df_crsp['shrcd']
del df_crsp['exchcd']
# Drop obs with missing returns
df_crsp = df_crsp[df_crsp.ret.notnull()]

# There are some negative number in the PRC column. Why is that? To find out, look at the doc!
# In our case, (in most cases actually), it's ok to just take the absolute value.
df_crsp['prc'] = np.abs(df_crsp['prc'])

In [ ]:
# Compute continuously compounded returns (i.e. log returns).
# Why are these useful?
df_crsp['lret'] = np.log(1 + df_crsp['ret'])
df_crsp['lvwretd'] = np.log(1 + df_crsp['vwretd'])
df_crsp['lewretd'] = np.log(1 + df_crsp['ewretd'])

# Compute the market cap
df_crsp['size'] = df_crsp['shrout'] * df_crsp['prc']

In [ ]:
# Look at the average size over time
df_crsp.groupby(['date'])['size'].mean().plot()

In [ ]:
# Total market size over time

ax = df_crsp.groupby(['date'])['size'].sum().plot()

In [ ]:
# More informative in logs!
ax = df_crsp.groupby(['date'])['size'].sum().plot(logy=True)

# Add some informative lines
ax.axvline(x=datetime(1929,10,24), color='k', linestyle=':') # Black Monday
ax.axvline(x=datetime(1987,10,19), color='k', linestyle=':') # Black Monday again
ax.axvline(x=datetime(2001,1,1), color='k', linestyle=':') # Tech bubble burst
ax.axvline(x=datetime(2008,9,16), color='k', linestyle=':')

## Step 2: analysis setup

In [ ]:
# Parameters
form_period = 36 # Formation period, in month
hold_period = 36 # Holding period, in months
use_deciles = True # Use deciles to form the top and bottom portfolio
n_stocks = 35  # Number of stocks in the top and bottom performance (if not using top and bottom deciles)
start_date = '1935-01-01'
end_date = '2014-01-01'
benchmark = 'vwretd' # Benchmark market return to use ('vwretd' or 'ewretd')

In [ ]:
# Get the dates of portfolio formation.
# The frequency tells how far appart to put the dates.
# 'M' stand for month, 'MS' is for month start, to make sure we 
# have first day of the month. It needs to be a string, so we convert
# our numbers to string. Note: you can also keep floats and use pandas
# date offsets.
dates = pd.date_range(start=start_date, end=end_date, freq=str(np.int(form_period)) + 'MS')
dates

## Step 3: forming portfolios

In this step, we want to create the portfolios at each portfolio formation date. Basically, we want to evaluate each stock's performance in the formation period prior to the formation date, and form portfolios (groups of stocks) based on past performance.

For simplicity, at each date we only keep track of the top and bottom portfolios.

In [ ]:
# Let's first do it for only one date.
date = dates[0]

beg_dt = date - pd.offsets.MonthBegin(1) * form_period

# Select obs for the formation period
crsp_t = df_crsp[beg_dt:date.to_pydatetime()].copy()

In [ ]:
crsp_t.describe()

In [ ]:
crsp_t.head()

In [ ]:
crsp_t.tail()

In [ ]:
# We only want to keep stocks that are there during the full formation window

crsp_t['N'] = crsp_t.groupby(['permno'])['permno'].transform('count')

# Filter on number of observations. We only keep sotcks for which we have returns
# over the full observation period.
crsp_t = crsp_t[crsp_t['N'] >= form_period]


In [ ]:
crsp_t.describe()

In [ ]:
# Now for each stock we want to compute the full period return. Easy with log returns, just sum up!
stock_ret = crsp_t.groupby('permno')['lret', 'lvwretd', 'lewretd'].sum()

In [ ]:
stock_ret.head()

In [ ]:
# Next compute excess returns based on the chosen index.
# Note that since the benchmark is the same for all stocks, we could use
# actual returns for ranking purposes. It would only make a difference in some
# cases. Which ones?

stock_ret['lexret'] = stock_ret['lret'] - stock_ret['l' + benchmark]

In [ ]:
# Now deciles and rankings.

stock_ret['decile'] = pd.qcut(stock_ret['lexret'], 10, labels=False) # (0=worst, 9=best)
stock_ret['rank_asc'] = stock_ret['lexret'].rank() # (1 = worst return)
stock_ret['rank_inv'] = stock_ret['lexret'].rank(ascending=False) # (1= best return)

In [ ]:
# Assign stock to top or bottom portfolio

if use_deciles:
    top_ptf = stock_ret[stock_ret.decile == 9].reset_index()[['permno', 'lexret']]
    bot_ptf = stock_ret[stock_ret.decile == 0].reset_index()[['permno', 'lexret']]
else:
    top_ptf = stock_ret[stock_ret.rank_inv <= n_stocks].reset_index()[['permno', 'lexret']]
    bot_ptf = stock_ret[stock_ret.rank_asc <= n_stocks].reset_index()[['permno', 'lexret']]
    

In [ ]:
top_ptf.head()

In [ ]:
bot_ptf.head()

Now that we have the code working for one date, we need to run it on each date. Now it's time to make it a function.

In [ ]:
def compute_performance_portfolios(date, df, form_period=36, use_deciles=True, n_stocks=35,
                                   benchmark='vwretd'):
    beg_dt = date - pd.offsets.MonthBegin(1) * form_period

    # Select obs for the formation period
    crsp_t = df[beg_dt:date.to_pydatetime()].copy()
    
    # We only want to keep stocks that are there during the full formation window
    crsp_t['N'] = crsp_t.groupby(['permno'])['permno'].transform('count')
    # Filter on number of observations. We only keep sotcks for which we have returns
    # over the full observation period.
    crsp_t = crsp_t[crsp_t['N'] >= form_period]
    
    # Now for each stock we want to compute the full period return. Easy with log returns, just sum up!
    stock_ret = crsp_t.groupby('permno')['lret', 'lvwretd', 'lewretd'].sum()# Next compute excess returns based on the chosen index.
    # Compute excess returns
    stock_ret['lexret'] = stock_ret['lret'] - stock_ret['l' + benchmark]
    # Now deciles and rankings.
    stock_ret['decile'] = pd.qcut(stock_ret['lexret'], 10, labels=False) # (0=worst, 9=best)
    stock_ret['rank_asc'] = stock_ret['lexret'].rank() # (1 = worst return)
    stock_ret['rank_inv'] = stock_ret['lexret'].rank(ascending=False) # (1= best return)
    # Assign stock to top or bottom portfolio
    if use_deciles:
        top_ptf = stock_ret[stock_ret.decile == 9].reset_index()[['permno', 'lexret']]
        bot_ptf = stock_ret[stock_ret.decile == 0].reset_index()[['permno', 'lexret']]
    else:
        top_ptf = stock_ret[stock_ret.rank_inv <= n_stocks].reset_index()[['permno', 'lexret']]
        bot_ptf = stock_ret[stock_ret.rank_asc <= n_stocks].reset_index()[['permno', 'lexret']]
    
    return (bot_ptf, top_ptf)

In [ ]:
portfolios = {}
for date in dates:
    portfolios[date] = compute_performance_portfolios(date, df_crsp, use_deciles=False, benchmark='vwretd')

In [ ]:
portfolios[date][0].head()

## Step 3: holding period returns

Next, we want to compute cumulative abnormal returns for portfolios during the holding period. We will use the same approach, i.e. do it for one portfolio/date, then package it as a function.

In [ ]:
date = dates[0]
ptf = portfolios[date][0] # Bottom portfolio.

benchmark = 'vwretd'
weighting = 'vw' # 'vw' or 'ew'


In [ ]:
ptf = ptf.copy()
end_dt = date + pd.offsets.MonthBegin(1) * hold_period

# Select obs for the formation period
crsp_t2 = df_crsp[date.to_pydatetime():end_dt].copy()

In [ ]:
# Merge with stocks in portfolios, to keep only those stocks
crsp_t2 = pd.merge(crsp_t2.reset_index(), ptf, on=['permno'])

In [ ]:
crsp_t2.head()

In [ ]:
crsp_t2.tail()

In [ ]:
crsp_t2.tail()

In [ ]:
# We want to make sure we have one observation for each stock/date.
# If a stock is delisted, its returns will be 0 after it disappears,
# so we just fill in these missing values.
# Note that here we're simplifying a bit. CRSP does include delisting
# returns that we should have added as well, but we didn't.

# The idea here is to create a DataFrame with all the permno/date pairs
# that we want in the final dataset. Then we merge that list with the
# dataset using "outer" which will generate missing values for the
# pairs that are not in the dataset.

# Get the dates in the dataset.
pairs_t2 = [{'date': d, 'permno': p} for d in crsp_t2['date'].unique() 
                                    for p in ptf['permno'].unique()]
pairs_t2[:10]

In [ ]:
pairs_t2 = pd.DataFrame(pairs_t2)
pairs_t2.head()

In [ ]:
crsp_t2.describe()

In [ ]:
# Merge to generate placeholders

crsp_t2 = pd.merge(crsp_t2, pairs_t2, how='outer',
                   on=['permno', 'date'])
crsp_t2.describe()

In [ ]:
ret_cols = ['ret', 'vwretd', 'ewretd', 'lvwretd','lewretd', 'lret', 'lexret']
crsp_t2[ret_cols] = crsp_t2[ret_cols].fillna(0.0)

In [ ]:
crsp_t2.describe()

In [ ]:
# Now we want the return up to each point in time
crsp_t2['lcumret'] = crsp_t2.groupby('permno')['lret'].cumsum()
crsp_t2['lcum' + benchmark] = crsp_t2.groupby('permno')['l' + benchmark].cumsum()

In [ ]:
# At each point in time, the return of the portfolio will be the 
# cumulative return of each component weighted by the initial weight.
# Note that here we need the simple return average, not log return.
crsp_t2['cumret'] = np.exp(crsp_t2['lcumret']) - 1
crsp_t2['cum' + benchmark] = np.exp(crsp_t2['lcum' + benchmark]) - 1

# Add weights, equal weighted is easy.
ptf['ew'] = 1 / len(ptf)

# For value-weighted, need to get size as of formation date.
ptf['date'] = date
weights = pd.merge_asof(ptf, df_crsp[['permno', 'size']],
                        by='permno',
                        left_on='date',
                        right_index=True)
weights['vw'] = weights['size'] / weights['size'].sum()

del weights['lexret']
del weights['date']
del weights['size']

In [ ]:
weights.head()

In [ ]:
# Now merge back with returns
crsp_t2 = pd.merge(crsp_t2, weights, on='permno')

In [ ]:
crsp_t2.head()

In [ ]:
# Now compute the weighted cumulative return
crsp_t2['wcumret'] = crsp_t2[weighting] * crsp_t2['cumret']
crsp_t2['wcum' + benchmark] = crsp_t2[weighting] * crsp_t2['cum' + benchmark]

ptf_ret = crsp_t2.groupby(['date'])[['wcumret', 'wcum' + benchmark]].sum()

In [ ]:
# Count the months
ptf_ret = ptf_ret.reset_index()
ptf_ret['months'] = ptf_ret.index.values + 1

In [ ]:
ptf_ret['exret'] = ptf_ret['wcumret'] - ptf_ret['wcum' + benchmark]

In [ ]:
ptf_ret.head()

Ok, looks like we're ready to package as a function.

In [ ]:
def compute_holding_returns(date, ptf, df, benchmark='vwretd', weighting='vw', hold_per=36):
    ptf = ptf.copy()
    end_dt = date + pd.offsets.MonthBegin(1) * hold_period
    # Select obs for the formation period
    crsp_t2 = df[date.to_pydatetime():end_dt].copy()
    # Merge with stocks in portfolios, to keep only those stocks
    crsp_t2 = pd.merge(crsp_t2.reset_index(), ptf, on=['permno'])
    

    # Get the dates in the dataset.
    pairs_t2 = [{'date': d, 'permno': p} for d in crsp_t2['date'].unique() 
                                        for p in ptf['permno'].unique()]
    pairs_t2 = pd.DataFrame(pairs_t2)
    crsp_t2 = pd.merge(crsp_t2, pairs_t2, how='outer',
                   on=['permno', 'date'])
    ret_cols = ['ret', 'vwretd', 'ewretd', 'lvwretd','lewretd', 'lret', 'lexret']
    crsp_t2[ret_cols] = crsp_t2[ret_cols].fillna(0.0)
    
    # Now we want the return up to each point in time
    crsp_t2['lcumret'] = crsp_t2.groupby('permno')['lret'].cumsum()
    crsp_t2['lcum' + benchmark] = crsp_t2.groupby('permno')['l' + benchmark].cumsum()

    # At each point in time, the return of the portfolio will be the 
    # cumulative return of each component weighted by the initial weight.
    # Note that here we need the simple return average, not log return.
    crsp_t2['cumret'] = np.exp(crsp_t2['lcumret']) - 1
    crsp_t2['cum' + benchmark] = np.exp(crsp_t2['lcum' + benchmark]) - 1

    # Add weights, equal weighted is easy.
    ptf['ew'] = 1 / len(ptf)

    # For value-weighted, need to get size as of formation date.
    ptf['date'] = date
    weights = pd.merge_asof(ptf, df_crsp[['permno', 'size']],
                            by='permno',
                            left_on='date',
                            right_index=True)
    weights['vw'] = weights['size'] / weights['size'].sum()

    del weights['lexret']
    del weights['date']
    del weights['size']
    
    # Now merge back with returns
    crsp_t2 = pd.merge(crsp_t2, weights, on='permno')
    
    # Now compute the weighted cumulative return
    crsp_t2['wcumret'] = crsp_t2[weighting] * crsp_t2['cumret']
    crsp_t2['wcum' + benchmark] = crsp_t2[weighting] * crsp_t2['cum' + benchmark]

    ptf_ret = crsp_t2.groupby(['date'])[['wcumret', 'wcum' + benchmark]].sum()
    
    # Count the months
    ptf_ret = ptf_ret.reset_index()
    ptf_ret['months'] = ptf_ret.index.values + 1
    
    ptf_ret['exret'] = ptf_ret['wcumret'] - ptf_ret['wcum' + benchmark]
    
    return ptf_ret

In [ ]:
bot_ptf_ret = []
top_ptf_ret = []

for date in dates:
    bot_ptf_ret.append(compute_holding_returns(date, portfolios[date][0], df_crsp,
                                               benchmark='vwretd', weighting='vw'))
    top_ptf_ret.append(compute_holding_returns(date, portfolios[date][1], df_crsp,
                                               benchmark='vwretd', weighting='vw'))
    
bot_ptf_ret = pd.concat(bot_ptf_ret)
top_ptf_ret = pd.concat(top_ptf_ret)

In [ ]:
ax = bot_ptf_ret.groupby('months')['exret'].mean().plot(label='Past losers')
top_ptf_ret.groupby('months')['exret'].mean().plot(ax=ax, label='Past winners')
ax.legend()
ax.axhline(y=0, color='black', alpha=0.5, linestyle=':')
ax.axvline(x=12, color='black', alpha=0.5, linestyle='-')
ax.axvline(x=24, color='black', alpha=0.5, linestyle='-')

In [ ]:
# Only pre 1980
ax = bot_ptf_ret.set_index('date')[:'1980'].groupby('months')['exret'].mean().plot(label='Past losers')
top_ptf_ret.set_index('date')[:'1980'].groupby('months')['exret'].mean().plot(ax=ax, label='Past winners')
ax.legend()
ax.axhline(y=0, color='black', alpha=0.5, linestyle=':')
ax.axvline(x=12, color='black', alpha=0.5, linestyle='-')
ax.axvline(x=24, color='black', alpha=0.5, linestyle='-')

In [ ]:
# Only post 1980
ax = bot_ptf_ret.set_index('date')['1980':].groupby('months')['exret'].mean().plot(label='Past losers')
top_ptf_ret.set_index('date')['1980':].groupby('months')['exret'].mean().plot(ax=ax, label='Past winners')
ax.legend()
ax.axhline(y=0, color='black', alpha=0.5, linestyle=':')
ax.axvline(x=12, color='black', alpha=0.5, linestyle='-')
ax.axvline(x=24, color='black', alpha=0.5, linestyle='-')

Exercise: check if the results chnage significantly if portfolio formation is not done in January.